In [9]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import getpass
from IPython.display import display, Image
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dve-shm-uage')

In [3]:
username = 'bxie43@wisc.edu'
password = 'Ben803803`'

In [4]:
master_idx = pd.read_csv("Master_sheet - Sheet1.csv")
company_list = master_idx['company_id_SPCIQ']

In [14]:
login_url = 'https://www.capitaliq.com/CIQDotNet/login-okta.aspx'


In [10]:
def page_login(username, password, url):
    bot = webdriver.Chrome(options=options)
    wait = WebDriverWait(bot, 20)
    bot.get(url)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#onetrust-accept-btn-handler"))).click()

    time.sleep(2)

    bot.find_element(By.ID, 'input28').send_keys(username)
    bot.find_element(By.ID, 'input28').send_keys(Keys.RETURN)

    time.sleep(2)

    password_field = bot.find_element(By.ID, 'input60')
    password_field.send_keys(password)
    password_field.send_keys(Keys.RETURN)

    return bot   

In [223]:
def get_company_data(bot, company_id, url, tag, idx):
    
    bot.get(url.format(company_id))

    df = None 
    
    try:
        table = bot.find_element(By.ID, tag)

        data = []
        for row in table.find_elements(By.TAG_NAME, "tr"):
            row_data = [cell.text for cell in row.find_elements(By.TAG_NAME, "td")]
            if row_data:
                data.append(row_data)
        headers = data[0]
        non_empty_headers = [header.strip() for header in headers if header]
        cleaned_rows = [[cell for cell in row if cell] for row in data[1:]]
        cleaned_rows = [row for row in cleaned_rows if any(cell for cell in row if cell)]

        if len(cleaned_rows) > 0: # Check if it has multiple rows
            try:
                df = pd.DataFrame(cleaned_rows[1:], columns=non_empty_headers) 
            except ValueError:
                try:
                    df = pd.DataFrame([cleaned_rows[0]], columns=non_empty_headers)  # Ensure this is a list of lists
                except Exception:
                    pass
        else: # Text extraction
            df = pd.DataFrame([table.text.replace('\n\n', ';')], columns = idx)
            
    except NoSuchElementException: # Data not disclosured
        df = pd.DataFrame([" "] * len(idx), index=idx).T
    
    # Only proceed if df is successfully created
    if df is not None:
        df.insert(0, 'company_id', str(company_id))
        if len(df[:-1]) != 0:
            df = df[:-1]

    return df

In [19]:
def merge_tables(tb1, tb2):
    return pd.concat([tb1, tb2], axis = 0)

In [227]:
bot = page_login(username, password, login_url)

#### 1. Customers

In [ ]:
customers_url = 'https://www.capitaliq.com/CIQDotNet/BusinessRel/Customers.aspx?CompanyId={}'
customers_index = ['Customer Name', 'Supplier Name', 'Relationship Type', 'Primary Industry', 'Source']
customers_tag = 'myCustomersGrid_gridSection_myDataGrid'

In [210]:
customers_table = pd.DataFrame()
for company_id in company_list:
    customers_table = merge_tables(customers_table, get_company_data(bot, company_id, customers_url, customers_tag, customers_index))

#### 2. Competitors

In [24]:
competitors_url = 'https://www.capitaliq.com/CIQDotNet/BusinessRel/Competitors.aspx?CompanyId={}'
competitors_index = ["Competitor's Name", 'Company', 'LTM Revenue ($mm)', 'LTM Date', 'Source']
competitors_tag = 'myCompetitorsGrid_gridSection_myDataGrid'

In [79]:
competitors_table = pd.DataFrame()
for company_id in company_list:
    competitors_table = merge_tables(competitors_table, get_company_data(bot, company_id, competitors_url, competitors_tag, competitors_index))

#### 3. Alliances

In [82]:
alliances_url = 'https://www.capitaliq.com/CIQDotNet/BusinessRel/StrategicAlliances.aspx?CompanyId={}'
alliances_index = ["Strategic Alliance Name", 'Company Name', 'Primary Industry', 'Source']
alliances_tag = 'myStrategicAlliancesGrid_gridSection_myDataGrid'

In [83]:
alliances_table = pd.DataFrame()
for company_id in company_list:
    alliances_table = merge_tables(alliances_table, get_company_data(bot, company_id, alliances_url, alliances_tag, alliances_index))

In [84]:
alliances_table

,company_id,Strategic Alliance Name,Company Name,Primary Industry,Source
0,251704,AGRANA Stärke GmbH,Archer-Daniels-Midland Company (NYSE:ADM),Packaged Foods and Meats,AGRANA Beteiligungs-Aktiengesellschaft (WBAG:A...
1,251704,Asia Sustainable Foods Platform,Archer-Daniels-Midland Company (NYSE:ADM),Archer-Daniels-Midland Company (NYSE:ADM) 2023...,None
2,251704,"Benson Hill, Inc. (NYSE:BHIL)",Archer-Daniels-Midland Company (NYSE:ADM),Agricultural Products and Services,Archer-Daniels-Midland Company (NYSE:ADM) 2023...
3,251704,"Cargill, Incorporated",Archer-Daniels-Midland Company (NYSE:ADM),Agricultural Products and Services,Archer-Daniels-Midland Company (NYSE:ADM) 2023...
4,251704,Devo Energy S.A. (WSE:DEV),"WILD Flavors, Inc.",Asset Management and Custody Banks,Devo Energy S.A. (WSE:DEV) - Form
...,...,...,...,...,...
1,190636,"Independent Purchasing Cooperative, Inc.","Tyson Foods, Inc. (NYSE:TSN)",Diversified Support Services,Key Development Strategic Alliances Nov-14-2018
2,190636,Malayan Flour Mills Berhad (KLSE:MFLOUR),"Tyson Foods, Inc. (NYSE:TSN)",Packaged Foods and Meats,Key Development Client Announcements Feb-10-2021
3,190636,Tanmiah Food Company (SASE:2281),"Tyson Foods, Inc. (NYSE:TSN)",Packaged Foods and Meats,Key Development Strategic Alliances Jul-06-2022
0,920226,"Morinaga Milk Industry Co., Ltd. (TSE:2264)",Thomas J Lipton Co.,Packaged Foods and Meats,"Morinaga Milk Industry Co., Ltd. (TSE:2264) 20..."


#### 4. Sourcing of Raw Materials

In [85]:
suppliers_url = 'https://www.capitaliq.com/CIQDotNet/BusinessRel/Suppliers.aspx?CompanyId={}'
suppliers_index = ["Supplier Name", 'Customer Name', 'Relationship Type', 'Primary Industry', 'Source']
suppliers_tag = 'mySuppliersGrid_gridSection_myDataGrid'

In [86]:
suppliers_table = pd.DataFrame()
for company_id in company_list:
    suppliers_table = merge_tables(suppliers_table, get_company_data(bot, company_id, suppliers_url, suppliers_tag, suppliers_index))

#### 5. Acquisitions

In [91]:
acquisitions_url = 'https://www.capitaliq.com/ciqdotnet/Transactions/TransactionSummary.aspx?CompanyId={}&transactionViewType=1'
acquisitions_index = ["Announced Date", 'Closed Date', 'Transaction Type', 'Role', 'Target', 'Buyer/Investors', 'Sellers',
                                                            'Rounds', 'Round Type', 'Pre-Money Valuation ($mm)', 'Post-Money Valuation ($mm)', 'Size ($mm)', 
                                                            'Aggregate Amount Raised ($mm)†']
acquisitions_tag = 'MATransactionGrid'

In [98]:
acquisitions_table = pd.DataFrame()
for company_id in company_list:
    acquisitions_table = merge_tables(acquisitions_table, get_company_data(bot, company_id, acquisitions_url, acquisitions_tag, acquisitions_index))

In [ ]:
acquisitions_table.to_csv('acquisitions.csv', index = False)

#### 6. Key Brands

In [157]:
brands_url = 'https://www.capitaliq.com/CIQDotNet/Company/LongBusinessDescription.aspx?CompanyId={}'
brands_index = ['Business Description and Products']
brands_tag = 'Displaysection1'

In [224]:
brands_table = pd.DataFrame()
for company_id in company_list:
    brands_table = merge_tables(brands_table, get_company_data(bot, company_id, brands_url, brands_tag, brands_index))

In [226]:
brands_table.to_csv('brands.csv', index=False)